# Introduction

Ragas is the de-facto opensource standard for RAG evaluations. Ragas provides features and methods to help evaluate RAG applications. In this notebook we will cover basic steps for evaluating your RAG application with Ragas. 

### Contents
- [Prerequisites]()
- [Dataset preparation]()
- [Evaluation]()
- [Analysis]()

### Prerequisites
- Ragas is a python package and we can install it using pip
- Ragas uses model guided techniques underneath to produce scores for each metric. In this tutorial, we will use OpenAI `gpt-3.5-turbo` and `text-embedding-ada-002`. These are the default models used in ragas but you can use any LLM or Embedding of your choice by referring to this [guide](https://docs.ragas.io/en/stable/howtos/customisations/bring-your-own-llm-or-embs.html). I highly recommend that you try this notebook with open-ai so that you get a feel of it with ease.

In [ ]:
! pip install ragas

In [ ]:
! export OPENAI_API_KEY="your-openai-key"

And that's it. You're ready to go.

## Dataset preparation

Evaluating any ML pipeline will require several data points that constitues a test dataset. For Ragas, the data points required for evaluating your RAG completely are

- `question`: A question or query that is relevant to your RAG.
- `contexts`: The retrieved contexts corresponding to each question. This is a `list[list]` since each question can retrieve multiple text chunks.
- `answer`:  The answer generated by your RAG corresponding to each question.
- `ground_truth`: The expected correct answer corresponding to each question.

For the purpose of this notebook, I have this dataset prepared from a simple RAG that I created myself to help me with NLP research. Let's use it.

In [ ]:
from datasets import load_dataset
PATH = ""

In [ ]:
eval_dataset = load_dataset(PATH)
eval_dataset

As you can see, the dataset contains all the required attributes mentioned above. Now we can move on our next step of actually doing the evaluation with it.

**Note:**
*We know that it's hard to formulate a test data containing Question and ground truth answer pairs when starting out. We have the perfect solution for this in this form of a ragas synthetic test data generation feature. The questions and ground truth answers were created by [ragas synthetic data generation]() feature. Check it out here once you finish this notebook*

## Evaluation
For evaluation ragas provides several metrics which is aimed to quantify the end-end performance of the pipeline and also the component wise performance of the pipeline. For this tutorial let's consider few of them

## Analysis